# Linear algebra in Julia
Author: Andreas Noack Jensen (MIT) (http://www.econ.ku.dk/phdstudent/noack/)

## Outline
 - Basic linear algebra 
 - Factorizations (Julia Innovation: Matrix Factorization Objects)
 - Special matrix Structures (Matrices know if they are triangular, tridiagonal, etc.)
 - Generic linear algebra (Increasingly Important and totally outside the scope of LAPACK)

### Basic linear algebra
Syntax very similar to other languages but there are some important differences. Define a matrix of random normal variates

In [ ]:
A = rand(1:4,3,3)

In [ ]:
B = A
C = copy(A)
[ B C]

In [ ]:
# Watch out!
A[1]=17
[B C] # B and A are references to the same memory (better for performance)

and a vector of ones

In [ ]:
x = ones(3)

Notice that $A$ has type Array{Float64,2} but $x$ has type Array{Float64,1}. Julia defines the aliases Vector{Type}=Array{Type,1} and Matrix{Type}=Array{Type,2} in Julia. 

Many of the basic operations are the same as in other languages
#### Multiplication

In [ ]:
b = A*x

#### Transposition
As in other languages `A'` is the conjugate transpose whereas `A.'` is just the transpose

In [ ]:
Asym = A + A'

#### Transposed multiplication
Julia allows us to write this without *

In [ ]:
Apd = A'A

#### Solving linear systems 
The problem $Ax=b$ for square $A$ is solved by the \ function.

In [ ]:
A\b

#### Overdetermined systems
When A is tall the \ function calculates the least squares solution.

In [ ]:
# Keep all rows and only the first 2 columns of `A` to generate `Atall`, representing an overdetermined linear system.
Atall = A[:,1:2]
display(Atall)
Atall\b

The \ function also works for rank deficient least squares problems. In this case, the least squares solution is not unique and Julia returns the solution with smallest norm.

In [ ]:
A = randn(3,3)

In [ ]:
# The outer product of a vector with itself will result in a singular matrix,
# representing a rank deficient least squares problem
[A[:,1] A[:,1]]\b

#### Underdetermined systems
when A is short the \ function returns the minimum norm solution.

In [ ]:
# Keep all columns and only the first 2 rows of `A` to generate `Ashort`, representing an underdetermined linear system.
Ashort = A[1:2,:]
display(Ashort)
Ashort\b[1:2]

### Factorization
The `\` function hides how the problem is actually solved. Depending on the dimensions of `A`, different methods are chosen to solve the problem. An intermediate step in the solution is to calculate a factorization of the matrix `A`. Basically, a factorization of `A` is a way of expressing `A` as a product of triangular, unitary and permutation matrices. Julia defines a `Factorization` abstract type and several composite subtypes for actually storing factorizations. A `Factorization` object should be thought of as a representation of the matrix `A`.

#### LU

When `A` is square, a linear system is solved by factorizing the matrix `PA=LU` where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular. Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [ ]:
A = randn(3,3)

In [ ]:
# One way to perform an LU factorization is with the function `lu`, which returns matrices l and u
# and permutation vector p.
l,u,p = lu(A)

In [ ]:
# Pivoting is on by default so we can't assume A == LU
display(norm(l*u - A))
display(norm(l*u - A[p,:]))

In [ ]:
# To turn off pivoting for LU factorizations, use the argument `Val{false}`
l,u,p = lu(A, Val{false})
norm(l*u - A)

In [ ]:
# A second way to perform an LU factorization is with the function `lufact`.
Alu = lufact(A)

The different parts of the factorization can be extracted by special indexing

In [ ]:
Alu[:P]

In [ ]:
Alu[:L]

In [ ]:
Alu[:U]

We can therefore compute the solution of $Ax=b$ from the factorization

In [ ]:
# PA = LU
# A = P'LU
# P'LUx = b
# LUx = Pb
# Ux = L\Pb
# x = U\L\Pb
Alu[:U]\(Alu[:L]\(Alu[:P]b))

However, more importantly the `LU` type allows dispatch and we can solve the system by

In [ ]:
Alu\b

This could be useful if the same left-hand-side is used for several right-hand-sides. The factorization can also be used for calculating the determinant because $\det(A)=\det(PLU)=\det(P)\det(U)=\pm \prod u_{ii}$ because $U$ is triangular and the sign is determined from $\det(P)$.

In [ ]:
det(A)

In [ ]:
det(Alu)

#### QR
When `A` is tall, Julia computes the least squares solution $\hat{x}$ that minimizes $\|Ax-b\|_2$. This can be done by factorizing $A=QR$ where $Q$ is unitary/orthogonal and $R=\left(\begin{smallmatrix}R_0\\0\end{smallmatrix}\right)$ and $R_0$ is upper triangular. With the QR factorization the minimum norm can be expressed
\begin{equation*}
\|Ax-b\|=\|QRx-b\|=\|Q(Rx-Q'b)\|=\|Rx-Q'b\|=\left\|\begin{pmatrix}R_0x-Q_0'b\\Q_1'b\end{pmatrix}\right\|=\|R_0x-Q_0'b\|+\|Q_1'b\|
\end{equation*}
and the problem therefore reduces to solving the square problem $R_0x=Q_0'b$ for $x$.

We can QR factorize the submatrix of the two first columns of $A$ by

In [ ]:
Aqr = qrfact(A[:,1:2])

Another feature of the QR factorizations is the `Q` types for storing the unitary matrices $Q$. They can be extracted from the `QR` types by indexing

In [ ]:
Aqr[:Q]

Similarly, the upper triangular matrix $R$ can be extracted by indexing

In [ ]:
# In this case, R is stored as a 2x2 matrix rather than as a 3x2
# because the last row of R is filled with 0's.
Aqr[:R]

Even though the matrix `Aqr[:Q]` is printed as a $3\times 3$ matrix in the factorization object, in practice it can represent the thin version as well. Hence both

In [ ]:
Aqr[:Q]*ones(2)

and

In [ ]:
Aqr[:Q]*ones(3)

works, but not

In [ ]:
Aqr[:Q]*ones(4)

The matrix has a compact internal representation, so indexing is only meaningful if you know how the factorization stores data.

In [ ]:
Aqr[:Q][1]

The QRCompactWY object `\` has a method for the QR and the least squares problem is therefore solved with

In [ ]:
Aqr\b

It should be noted that this is *not* the way `A[:,1:2]\b` is solved. Instead Julia defaults to QR factorizations with pivoting in order to be able to handle rank deficient problems . Pivoting is enabled with a keyword

In [ ]:
Aqrp = qrfact([A[:,1] A[:,1]],Val{true})

Notice that the type is different now. `\` also has a method for `QRPivoted` and the rank deficient problem is therefore computed

In [ ]:
Aqrp\b

#### Eigendecompositions and the SVD(s)

The results from eigendecompositions and singular value decompositions are also stored in `Factorization` types. This also includes Hessenberg and Schur factorizations.

The eigendecomposition can be computed

In [ ]:
AsymEig = eigfact(Asym)

The values and the vectors can be extracted from the Eigen type by special indexing

In [ ]:
AsymEig[:values]

In [ ]:
AsymEig[:vectors]

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [ ]:
inv(AsymEig)*Asym

Julia also has an `eig` function which returns a tuple with the values and the vectors

In [ ]:
eig(Asym)

We do not recommend this version.

The `svdfact` function computes the singular value decomposition

In [ ]:
Asvd = svdfact(A[:,1:2])

and again `\` has a method for the type enabling least squares by SVD

In [ ]:
Asvd\b

There are special functions for providing values only: `eigvals` and `svdvals`.

### Special matrix Structures
The structure of matrices is very important in linear algebra. This structure can be made explicit in Julia through composite types. Examples are `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`. Specialized methods are written for the special matrix types to take advantage of their structure. Below some examples are shown

In [ ]:
A

In [ ]:
Diagonal(diag(A))

In [ ]:
Diagonal(A)

In [ ]:
# Note that this gives the same result as LowerTriangular(tril(A))
LowerTriangular(A)

In [ ]:
Symmetric(Asym)

In [ ]:
SymTridiagonal(diag(Asym),diag(Asym,1))

When it is known that a matrix is e.g. triangular or symmetric Julia might be able to solve a problem faster by converting the matrix to a special matrix. For some of the procedures, Julia checks if the input matrix is triangular or symmetric and converts the matrix if such a structure is detected. It should be noted that `Symmetric`, `Hermitian` and `Triangular` do not copy the input matrix.

#### Symmetric eigenproblem
Whether or not Julia is able to detect if a matrix is symmetric/Hermitian can have a big influence on how fast an eigenvalue problem is solved. Sometimes it is known that a matrix is symmetric or Hermitian but due to floating point errors this is not detected by the `eigvals` function. In following example `Asym1` and `Asym2` are almost identical, but unless Julia is told that `Asym2` is symmetric, the elapsed time for the computation is very different.

In [ ]:
n = 1000;
A = randn(n,n);
Asym1 = A + A';
Asym2 = copy(Asym1); Asym2[1,2] += 5eps();
println("Is Asym1 symmetric? ", issymmetric(Asym1))
println("Is Asym2 symmetric? ", issymmetric(Asym2))

In [ ]:
@time eigvals(Asym1);

In [ ]:
@time eigvals(Asym2);

In [ ]:
@time eigvals(Symmetric(Asym2));

### A big problem
Using tridiagonal matrices makes it possible to work with potentially very large problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a `Matrix` type.

In [ ]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

### Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does. For a long time Julia has also had support for multiplicaton of general element types. Hence, when multiplying integer matrices, the output is also an integer matrix

In [ ]:
rand(1:10,3,3)*rand(1:10,3,3)

Recently, more generic linear algebra methods has been added and Julia now supports generic `LU` and `QR` factorizations. Generic eigenvalue and SVD methods have been written more recently (some in external packages).

In general, the `LU` factorization can be computed whenever the matrix element type is closed under the operations `+`, `-`, `*` and `\`. Of course the matrix also has to have full rank. The generic `LU` method in Julia applies pivoting and therefore the element type also has to support `<` and `abs`. Hence it is possible to solve systems of equations of e.g. rational numbers which the following examples show.

#### Example 1: Rational linear system of equations
Julia has rational numbers built in. The following example shows how a linear system of equations can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [ ]:
Ar = convert(Matrix{Rational{BigInt}}, rand(1:10,3,3))/10

In [ ]:
x = ones(Int,3)
b = Ar*x

In [ ]:
Ar\b

In [ ]:
lufact(Ar)

#### Example 2: Rational matrix from eigenstructure
The next example shows how rational matrix arithmetic can be used for calculating a matrix given rational eigenvalues and -vectors. I have found this convenient when giving examples of linear dynamic systems.

In [ ]:
λ1,λ2,λ3 = 1//1,1//2,1//4
v1,v2,v3 = [1,0,0],[1,1,0],[1,1,1]
V,Λ = [v1 v2 v3], Diagonal([λ1,λ2,λ3])
A = V*Λ/V